Main reference:

https://medium.com/swlh/natural-language-processing-nlp-analysis-with-amazon-review-data-part-i-data-engineering-6573b782e4dc
https://melaniesoek0120.medium.com/natural-language-processing-nlp-amazon-review-data-part-ii-eda-data-preprocessing-and-model-3866dcbdbb77 


## Read and DropNan

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer as ps
from nltk.stem.wordnet import WordNetLemmatizer


df = pd.read_csv(r"C:\Users\Lim Jia Hui\Desktop\songsdata.csv")
print(len(df.reviewText))
print(df.head())
nanlist = []
for text in range(len(df.reviewText)):
    if type(df.reviewText[text])!= str:
        nanlist.append(int(text))
print(nanlist)
df.drop(index=nanlist, axis=0,inplace=True)
df.reset_index(inplace=True)


93907
   overall                                         reviewText
0        5  This is awesome to listen to, A must-have for ...
1        5                                               bien
2        5  It was great to hear the old stuff again and I...
3        4  well best of's are a bit poison normally but t...
4        5  What can I say? This is Casting Crowns!!!This ...
[2165, 54714]


# Feature Extraction

## Stopwords

In [2]:
sw_list = stopwords.words('english')
sw_list += list(string.punctuation)
sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘',"'", '©','said',"'s", "also",'one',"n't",'com', '-', '–', '—', '_',"/"]

## Tokenizer, Stemmer, and Lemmatizer

In [3]:
def Tokenizer(data):
    tokens = nltk.word_tokenize(data)
    tokenized_data= []
    for i in tokens:
        if i.lower() not in sw_list:
            tokenized_data.append(i.lower())
    return tokenized_data

def Stemmer(data2):
    stemmed_data =[]
    for j in data2:
        stemmed_data.append(ps().stem(j))
    return stemmed_data

def Lemmatizer(data3):
    lemmatized_data = []
    for k in data3:
        lemmatized_data.append(WordNetLemmatizer().lemmatize(k, pos='v'))
    return lemmatized_data



lemmatized_reviews = list(map(Lemmatizer,(map(Stemmer, (map(Tokenizer, df['reviewText']))))))

print(lemmatized_reviews[:5])



[['awesom', 'listen', 'must-hav', 'slayer', 'fan', '..', 'sadli', 'need', 'tripl', 'disc', 'set', '..', 'mani', 'hit'], ['bien'], ['great', 'hear', 'old', 'stuff', 'like', 'new', 'stuff', 'recommend', 'slayer', 'fan'], ['well', 'best', 'bite', 'poison', 'normal', 'bad', 'pretti', 'good', "'d", 'put', '90', 'hell', 'await', 'reign', 'blood', 'south', 'season', 'divin', 'coupl', 'musica', 'track', 'everyth', 'god', 'hate', '-at', 'point', 'best', 'mean', 'everi', 'cd', 'mainli', 'bad', 'dose', 'put', 'great', 'track', 'live', 'show', 'play', 'much', 'like,213', 'skeleton', 'societi', 'sex', 'murder', 'art', 'gemini', 'rare', 'track', 'final', 'six', 'bonu', 'track', 'christ', 'illus', 'mysteri', 'cover', 'song', 'unditstput', 'attitud', 'cd', 'would', 'greatest', 'hit', 'collect', 'know', 'put', 'coupl', 'live', 'track', 'too.al', 'could', 'much', 'wors', 'great', 'car'], ['say', 'cast', 'crown', 'good', 'bless', 'fill', 'cd']]


## Post Lemmatized Nan Drops

In [4]:
nanlist2 = []
for word in range(len(lemmatized_reviews)):
    if len(lemmatized_reviews[word]) == 0:
        nanlist2.append(word)
print(nanlist2)
print(len(nanlist2))
print(len(lemmatized_reviews))
for nan in nanlist2:
    lemmatized_reviews.remove([])
print(len(lemmatized_reviews))



[293, 1792, 1942, 2652, 3560, 3857, 5025, 5122, 5568, 6552, 7846, 8136, 9464, 10069, 11448, 11803, 12422, 12825, 13354, 14932, 18283, 28588, 30450, 32356, 32601, 33455, 35223, 35698, 35703, 37831, 37910, 39228, 40752, 42037, 43383, 44559, 45716, 48627, 50097, 52295, 52380, 52690, 56502, 59520, 60623, 61150, 61165, 62528, 65639, 66416, 69224, 70043, 70892, 71015, 71677, 72711, 73298, 73509, 73897, 75327, 75406, 76099, 77468, 77836, 77867, 80751, 81848, 82071, 82843, 85562, 86105, 86398, 88067, 88161, 88665, 89174, 89212, 90531, 91580, 92817]
80
93905
93825


### Removal from original df

In [5]:
df.drop(index=nanlist2, axis=0,inplace=True)
df.reset_index(inplace=True)


## Train-Test and Train-Val-Test Split

In [6]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(pd.DataFrame({'processed_reviews':lemmatized_reviews}), df['overall'].to_frame(name='overall'), test_size=0.2, random_state=0)

x_train_train, x_val, y_train_train, y_val = train_test_split(x_train,y_train,test_size=0.2,stratify=y_train, random_state=0)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

print(x_train_train.shape)
print(x_val.shape)
print(y_train_train.shape)
print(y_val.shape)



(75060, 1)
(18765, 1)
(75060, 1)
(18765, 1)
(60048, 1)
(15012, 1)
(60048, 1)
(15012, 1)


## Resampling


In [7]:
# optional "manual" resampler with sklearn.utils.resample

# from sklearn.utils import resample

# xy_train = pd.concat([x_train,y_train], axis=1)
# xy_train_train = pd.concat([x_train_train,y_train_train], axis=1)

# def resampling(minority):
#    resampled_minority = resample(minority, replace=True, n_samples= len(xy_train[xy_train.overall==5]), random_state=0)
#    return resampled_minority

# resampled_xy_train = xy_train[xy_train.overall==5]
# resampled_xy_train_train = xy_train_train[xy_train_train.overall==5]

# for i in range (1,5):
#    resampled_xy_train = pd.concat([resampled_xy_train,resampling(xy_train[xy_train.overall==i])])

# for i in range (1,5):
#    resampled_xy_train_train = pd.concat([resampled_xy_train_train,resampling(xy_train_train[xy_train_train.overall==i])])

# resampled_shuffled_xy_train = resampled_xy_train.sample(frac=1)
# x_train = resampled_shuffled_xy_train['processed_reviews'].to_frame()
# y_train = resampled_shuffled_xy_train['overall'].to_frame()

# resampled_shuffled_xy_train_train = resampled_xy_train_train.sample(frac=1)
# x_train_train = resampled_shuffled_xy_train_train['processed_reviews'].to_frame()
# y_train_train = resampled_shuffled_xy_train_train['overall'].to_frame()

In [8]:
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler

x_train, y_train = RandomOverSampler(random_state=0).fit_resample(x_train,y_train)

x_train_train, y_train_train = RandomOverSampler(random_state=0).fit_resample(x_train_train,y_train_train)



## Tf-Idf Vectorization

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

def dummy_token_and_pre(doc):
    return doc

tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_token_and_pre, preprocessor=dummy_token_and_pre, token_pattern=None)

tfidf_val = TfidfVectorizer(analyzer='word', tokenizer=dummy_token_and_pre, preprocessor=dummy_token_and_pre, token_pattern=None)

x_trainvec = tfidf.fit_transform(x_train.processed_reviews).sorted_indices()
x_testvec = tfidf.transform(x_test.processed_reviews).sorted_indices()

x_train_trainvec = tfidf_val.fit_transform(x_train_train.processed_reviews).sorted_indices()
x_valvec = tfidf_val.transform(x_val.processed_reviews).sorted_indices()

## Stacking

### TF MLP

In [10]:
#pseudocode for mlp hyperparam tuning

#obj()
    #param = suggest()

    #model = model()

    #for epochs in range()
    #            agg_f1 = []

    #    for train_index, val_index in StratifiedKFold(n_splits=5).split(x_trainvec,y_train): 

    #        x_trainvec_resd, y_train_resd = RandomOverSampler(random_state=0).fit_resample(x_trainvec.iloc[train_index], y_train.iloc[train_index])    

    #        model.fit(x=x_trainvec_resd, y=y_train_resd, epochs =(5^(rung)), verbose=0, batch_size= 100, random_state=0, callbacks=es)

    #        agg_f1.append(f1_score(y_pred=model.predict(pd.DataFrame.sparse.from_spmatrix(x_trainvec).iloc[val_index]), y_true=y_train.iloc[val_index].to_numpy().ravel(), average = 'macro'))

    #    intermediate_value = statistics.mean(agg_f1)



#rant: The challenge in implementing cross-validation for a tf model is the use of a tf model with (a lack of)sklearn functionalities
#      In this case, tf models have some hyperparams that need do be defined in .fit(), which clashes with sklearn functionalities 
#      as some sklearn functions have .fit() implicit in them. The important one here is cross_val_score(). Without this, one has to
#      resort to manually implementing the (stratified)cross_val function. This is a further issue when the data is imbalanced.
#      From googling it seems that the way to treat imbalanced data in cross-validation is to oversample the training folds
#      for each cross-validation split. This can normally be combined into a sklearn/imblearn pipeline and then passed to the
#      cross-validation function as seen in the random forest model. However, as mentioned this is a tf model with sklearn functionalities,
#      hence one has to implement this part manually as well. This is a bigger conundrum that one might initially expect.
#      As the method used is usually to slice out the training part of each cross-validation fold and then apply an oversampling function.
#      However, slicing a sparse matrix with a 1d np array as input is something that googling doesnt show how to do.
#      Now, you might think of transforming the sparse matrix into a pd df so that you could slice it with say iloc, but  
#      because of an unknown reason while the oversampling function as mentioned previously is supposed to be able to take 
#      both pandas df and sparse matrix as inputs, the pandas df version of a sparse matrix is not a valid input. This is vile shenanigans 
#      and I am utterly repulsed by it. Yet, given all this there is still the option of moving the tf-idf vectorization into the 
#      cross-validation part so that the sparse matrix is only made post oversampling, or making your own cross-validation
#      function with libraries using an updated version of pandas/scipy(both of which I will NOT do).
#  

#      Now one might be wondering why one should use a tf model anyways given its imperfect compatabiities(at least in cross-validation).  
#      The answer is GPU. Sklearn does not offer GPU support for training its models. The increase in training speed given GPU support is  
#      highly considerable. However, given that cross-validation is such a pain to implement, one might ask whether its still worth using even
#      if you have to redo the tf-idf vectorization every split for every epoch for every set of hyperparameters. The answer is something I do 
#      not know and will not attempt to know. Hence, the question is then rephrased into whether the extra speed from using the GPU is worth 
#      the decrease in samples used for the training set as some samples need to be separated into a validation set. Admittedly, I personally 
#      answered this question rather arbitrarily and perhaps personally biased to the superiority of Optuna over sklearn hyperparameter tuning.
#      Hence, I can't exactly justify my choice here, and what I do here is best treated just as a proof of concept.
            


In [11]:
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.optimizer_v2.gradient_descent import SGD
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
import statistics
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from tensorflow_addons.metrics import F1Score
from keras.utils import to_categorical
import scipy



def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

tf_x_train_trainvec = convert_sparse_matrix_to_sparse_tensor(x_train_trainvec)

tf_x_valvec = convert_sparse_matrix_to_sparse_tensor(x_valvec)

def create_model(trial, X):
    lr = trial.suggest_uniform('lr', 0.00001, 0.1)
    units = trial.suggest_int('units', 10, 50)
    input_dim = X

    model = Sequential()
    model.add(Dense(units, input_dim=input_dim, activation = 'relu', kernel_initializer='he_uniform'))
    model.add(Dense(5, activation='softmax'))

    optimizer = SGD(learning_rate=lr, momentum=0.9)

    

    model.compile(loss='categorical_crossentropy', 
                    optimizer=optimizer, 
                    metrics=['accuracy', F1Score(num_classes=5, average= 'macro', name='tf_f1')])
    return model
            

def objective_mlp(trial):

    model = create_model(trial, x_train_trainvec.shape[1])

    callbacks = [EarlyStopping(monitor='val_tf_f1', mode='max', min_delta=0.01, patience=3, verbose=1), TFKerasPruningCallback(trial,'val_tf_f1')]

    fit_model = model.fit(x=tf_x_train_trainvec, y=to_categorical(y_train_train.sub(1)), epochs =50, verbose=1, batch_size= 50, callbacks=callbacks, validation_data=(tf_x_valvec,to_categorical(y_val.sub(1))))

    return fit_model.history['val_tf_f1'][-1]

def show_result(study):

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    trial_idx = 0
    for frozen_trial in study.get_trials(deepcopy=False):

        print("Trial {} completed rungs: {}".format(trial_idx, frozen_trial.system_attrs))
        trial_idx +=1

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

study = optuna.create_study(
    direction="maximize",sampler= TPESampler(),  pruner=SuccessiveHalvingPruner()
)
study.optimize(objective_mlp, n_trials=100)

show_result(study)



[I 2022-08-25 03:06:54,025] A new study created in memory with name: no-name-a418ea7d-74b9-47ef-bc88-9940be52076b


Epoch 1/50


c:\Users\Lim Jia Hui\Desktop\Python\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 19), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


4644/4644 [==============================] - 24s 5ms/step - loss: 0.8447 - accuracy: 0.6544 - tf_f1: 0.5249 - val_loss: 1.1850 - val_accuracy: 0.5142 - val_tf_f1: 0.6462
Epoch 2/50
4644/4644 [==============================] - 23s 5ms/step - loss: 0.5425 - accuracy: 0.7843 - tf_f1: 0.6830 - val_loss: 1.0550 - val_accuracy: 0.5982 - val_tf_f1: 0.7111
Epoch 3/50
4644/4644 [==============================] - 24s 5ms/step - loss: 0.4656 - accuracy: 0.8165 - tf_f1: 0.7289 - val_loss: 1.1352 - val_accuracy: 0.6232 - val_tf_f1: 0.7440
Epoch 4/50
4644/4644 [==============================] - 23s 5ms/step - loss: 0.4180 - accuracy: 0.8364 - tf_f1: 0.7552 - val_loss: 1.3025 - val_accuracy: 0.4760 - val_tf_f1: 0.7643
Epoch 5/50
4644/4644 [==============================] - 23s 5ms/step - loss: 0.3860 - accuracy: 0.8504 - tf_f1: 0.7718 - val_loss: 1.1065 - val_accuracy: 0.6535 - val_tf_f1: 0.7793
Epoch 6/50
4644/4644 [==============================] - 23s 5ms/step - loss: 0.3629 - accuracy: 0.8604 - t

[I 2022-08-25 03:13:04,037] Trial 0 finished with value: 0.8416303396224976 and parameters: {'lr': 0.07652462285473018, 'units': 19}. Best is trial 0 with value: 0.8416303396224976.


...

Epoch 1/50


c:\Users\Lim Jia Hui\Desktop\Python\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_99/dense_198/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_99/dense_198/embedding_lookup_sparse/Reshape:0", shape=(None, 49), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_99/dense_198/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


4644/4644 [==============================] - 24s 5ms/step - loss: 0.7773 - accuracy: 0.6788 - tf_f1: 0.5493 - val_loss: 1.1152 - val_accuracy: 0.5157 - val_tf_f1: 0.6728
Epoch 2/50
4644/4644 [==============================] - 24s 5ms/step - loss: 0.4204 - accuracy: 0.8282 - tf_f1: 0.7148 - val_loss: 1.2379 - val_accuracy: 0.4927 - val_tf_f1: 0.7450


[I 2022-08-25 06:19:13,749] Trial 99 pruned. Trial was pruned at epoch 1.


Trial 0 completed rungs: {}
Trial 1 completed rungs: {'completed_rung_0': 0.7427323460578918, 'completed_rung_1': 0.8189261555671692, 'completed_rung_2': 0.8944036960601807}
Trial 2 completed rungs: {'completed_rung_0': 0.6741511821746826}
Trial 3 completed rungs: {'completed_rung_0': 0.7349013090133667}
Trial 4 completed rungs: {'completed_rung_0': 0.7399435639381409}
Trial 5 completed rungs: {'completed_rung_0': 0.5655180215835571}
Trial 6 completed rungs: {'completed_rung_0': 0.738447368144989}
Trial 7 completed rungs: {'completed_rung_0': 0.7282981276512146}
Trial 8 completed rungs: {'completed_rung_0': 0.7315474152565002}
Trial 9 completed rungs: {'completed_rung_0': 0.7375059723854065}
Trial 10 completed rungs: {'completed_rung_0': 0.7100772857666016}
Trial 11 completed rungs: {'completed_rung_0': 0.727260947227478}
Trial 12 completed rungs: {'completed_rung_0': 0.6876068115234375}
Trial 13 completed rungs: {'completed_rung_0': 0.6354276537895203}
Trial 14 completed rungs: {'comp

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

best_model = create_model(study.best_trial, x_trainvec.shape[1])

tf_x_trainvec = convert_sparse_matrix_to_sparse_tensor(x_trainvec)
tf_x_testvec = convert_sparse_matrix_to_sparse_tensor(x_testvec)

best_model.fit(x=tf_x_trainvec, y=to_categorical(y_train.sub(1)), epochs =50, verbose=1, batch_size= 50, callbacks=EarlyStopping(monitor='tf_f1', mode='max', min_delta=0.01, patience=3, verbose=1))

mlp_train_predictions = np.argmax(best_model.predict(x_trainvec),axis=1)+(1)
mlp_test_predictions = np.argmax(best_model.predict(x_testvec),axis=1)+(1)

Epoch 1/50


c:\Users\Lim Jia Hui\Desktop\Python\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_100/dense_200/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_100/dense_200/embedding_lookup_sparse/Reshape:0", shape=(None, 49), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_100/dense_200/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


5805/5805 [==============================] - 29s 5ms/step - loss: 0.7832 - accuracy: 0.6778 - tf_f1: 0.5581
Epoch 2/50
5805/5805 [==============================] - 29s 5ms/step - loss: 0.4407 - accuracy: 0.8202 - tf_f1: 0.7155
Epoch 3/50
5805/5805 [==============================] - 29s 5ms/step - loss: 0.3468 - accuracy: 0.8588 - tf_f1: 0.7670
Epoch 4/50
5805/5805 [==============================] - 29s 5ms/step - loss: 0.2975 - accuracy: 0.8806 - tf_f1: 0.7966
Epoch 5/50
5805/5805 [==============================] - 29s 5ms/step - loss: 0.2649 - accuracy: 0.8952 - tf_f1: 0.8170
Epoch 6/50
5805/5805 [==============================] - 29s 5ms/step - loss: 0.2419 - accuracy: 0.9064 - tf_f1: 0.8323
Epoch 7/50
5805/5805 [==============================] - 29s 5ms/step - loss: 0.2227 - accuracy: 0.9153 - tf_f1: 0.8444
Epoch 8/50
5805/5805 [==============================] - 29s 5ms/step - loss: 0.2072 - accuracy: 0.9227 - tf_f1: 0.8544
Epoch 9/50
5805/5805 [==============================] - 29s

### MLP Evaluation

In [15]:
print("MLP Train Accuracy Score :",accuracy_score(y_pred=mlp_train_predictions, y_true=y_train.to_numpy().ravel()))
print("MLP Test Accuracy Score :",accuracy_score(y_pred=mlp_test_predictions, y_true= y_test.to_numpy().ravel()))

print("MLP Train F1 Score :",f1_score(y_pred=mlp_train_predictions, y_true=y_train.to_numpy().ravel(), average = 'macro'))
print("MLP Test F1 Score :",f1_score(y_pred=mlp_test_predictions, y_true=y_test.to_numpy().ravel(), average = 'macro'))

print("MLP Confusion matrix:\n{}".format(confusion_matrix(y_pred=y_test.to_numpy().ravel(),y_true= mlp_test_predictions)))
print("MLP Classification report:\n",classification_report(y_pred=mlp_test_predictions,  y_true= y_test.to_numpy().ravel()))

MLP Train Accuracy Score : 0.9579821508562765
MLP Test Accuracy Score : 0.6490807354116707
MLP Train F1 Score : 0.9578983028224213
MLP Test F1 Score : 0.36164370916557304
MLP Confusion matrix:
[[  112    31    54    58   205]
 [   34    56    53    51    97]
 [   45    73   262   312   607]
 [   28    41   216   936  2796]
 [   74    65   292  1453 10814]]
MLP Classification report:
               precision    recall  f1-score   support

           1       0.24      0.38      0.30       293
           2       0.19      0.21      0.20       266
           3       0.20      0.30      0.24       877
           4       0.23      0.33      0.27      2810
           5       0.85      0.74      0.79     14519

    accuracy                           0.65     18765
   macro avg       0.34      0.39      0.36     18765
weighted avg       0.71      0.65      0.67     18765



### Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline


rfc = RandomForestClassifier(random_state=0, n_jobs=-1)

rfcgrid = {
    'rfc__max_depth' : list(range(1,50,5)),
    'rfc__n_estimators' : list(range(1,500,50)),
}

f1 = make_scorer(f1_score, average='macro') 

imba_pipe_rfc = Pipeline([
    ('sampling', RandomOverSampler(random_state=0)), 
    ('rfc', rfc)
    ])


rfcrs = HalvingGridSearchCV(estimator=imba_pipe_rfc, param_grid=rfcgrid, factor=5, cv=5, scoring=f1, verbose=10)

rfcrs.fit(x_trainvec, y_train.to_numpy().ravel())

print(rfcrs.best_score_)
print(rfcrs.best_params_)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 11608
max_resources_: 290210
aggressive_elimination: False
factor: 5
----------
iter: 0
n_candidates: 100
n_resources: 11608
Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5; 1/100] START rfc__max_depth=1, rfc__n_estimators=1.....................
[CV 1/5; 1/100] END rfc__max_depth=1, rfc__n_estimators=1;, score=(train=0.092, test=0.088) total time=   1.3s
[CV 2/5; 1/100] START rfc__max_depth=1, rfc__n_estimators=1.....................
[CV 2/5; 1/100] END rfc__max_depth=1, rfc__n_estimators=1;, score=(train=0.092, test=0.091) total time=   0.5s
[CV 3/5; 1/100] START rfc__max_depth=1, rfc__n_estimators=1.....................
[CV 3/5; 1/100] END rfc__max_depth=1, rfc__n_estimators=1;, score=(train=0.093, test=0.090) total time=   0.5s
[CV 4/5; 1/100] START rfc__max_depth=1, rfc__n_estimators=1.....................
[CV 4/5; 1/100] END rfc__max_depth=1, rfc__n_estimators=1;, score=(train=

### RFC Evaluation

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

rfc_train_predictions = rfcrs.best_estimator_.predict(x_trainvec)
rfc_test_predictions = rfcrs.best_estimator_.predict(x_testvec)

print("RFC Train Accuracy Score :",accuracy_score(y_pred=rfc_train_predictions, y_true=y_train.to_numpy().ravel()))
print("RFC Test Accuracy Score :",accuracy_score(y_pred=rfc_test_predictions, y_true= y_test.to_numpy().ravel()))

print("RFC Train F1 Score :",f1_score(y_pred=rfc_train_predictions, y_true=y_train.to_numpy().ravel(), average = 'macro'))
print("RFC Test F1 Score :",f1_score(y_pred=rfc_test_predictions, y_true=y_test.to_numpy().ravel(), average = 'macro'))

print("RFC Confusion matrix:\n{}".format(confusion_matrix(y_pred=y_test.to_numpy().ravel(),y_true= rfc_test_predictions)))
print("RFC Classification report:\n",classification_report(y_pred=rfc_test_predictions,  y_true= y_test.to_numpy().ravel()))

RFC Train Accuracy Score : 0.8295165569759829
RFC Test Accuracy Score : 0.7552358113509192
RFC Train F1 Score : 0.8369057182728585
RFC Test F1 Score : 0.4101705085267285
RFC Confusion matrix:
[[   99    20    28    19   109]
 [    7    37     7     1     3]
 [   13    40   186    96   130]
 [   19    45   233   870  1297]
 [  155   124   423  1824 12980]]
RFC Classification report:
               precision    recall  f1-score   support

           1       0.36      0.34      0.35       293
           2       0.67      0.14      0.23       266
           3       0.40      0.21      0.28       877
           4       0.35      0.31      0.33      2810
           5       0.84      0.89      0.86     14519

    accuracy                           0.76     18765
   macro avg       0.52      0.38      0.41     18765
weighted avg       0.73      0.76      0.74     18765



### Multinomial Naive Bayes Classifier

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

mnbc = MultinomialNB()

mnbc.get_params()

mnbc_grid = {
    'mnbc__alpha' : [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
    'mnbc__fit_prior' : [True, False]
}

imba_pipe_mnbc = Pipeline([
    ('sampling', RandomOverSampler(random_state=0)), 
    ('mnbc', mnbc)
    ])

f1 = make_scorer(f1_score, average='weighted')

mnbcgs = GridSearchCV(estimator=imba_pipe_mnbc, param_grid=mnbc_grid, cv=5, scoring=f1, verbose=10)

mnbcgs.fit(x_trainvec, y_train.to_numpy().ravel())

print(mnbcgs.best_score_)
print(mnbcgs.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5; 1/20] START mnbc__alpha=0.1, mnbc__fit_prior=True......................
[CV 1/5; 1/20] END mnbc__alpha=0.1, mnbc__fit_prior=True;, score=0.738 total time=   0.1s
[CV 2/5; 1/20] START mnbc__alpha=0.1, mnbc__fit_prior=True......................
[CV 2/5; 1/20] END mnbc__alpha=0.1, mnbc__fit_prior=True;, score=0.743 total time=   0.1s
[CV 3/5; 1/20] START mnbc__alpha=0.1, mnbc__fit_prior=True......................
[CV 3/5; 1/20] END mnbc__alpha=0.1, mnbc__fit_prior=True;, score=0.738 total time=   0.1s
[CV 4/5; 1/20] START mnbc__alpha=0.1, mnbc__fit_prior=True......................
[CV 4/5; 1/20] END mnbc__alpha=0.1, mnbc__fit_prior=True;, score=0.743 total time=   0.1s
[CV 5/5; 1/20] START mnbc__alpha=0.1, mnbc__fit_prior=True......................
[CV 5/5; 1/20] END mnbc__alpha=0.1, mnbc__fit_prior=True;, score=0.739 total time=   0.1s
[CV 1/5; 2/20] START mnbc__alpha=0.1, mnbc__fit_prior=False.....................
[C

### MNBC Evaluation

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

mnb_train_predictions = mnbcgs.best_estimator_.predict(x_trainvec)
mnb_test_predictions = mnbcgs.best_estimator_.predict(x_testvec)

print("MNBC Model Train Accuracy Score :",accuracy_score(mnb_train_predictions, y_train.to_numpy().ravel()))
print("MNBC Model Test Accuracy Score :",accuracy_score(mnb_test_predictions, y_test.to_numpy().ravel()))

print("MNBC Model Train F1 Score :",f1_score(mnb_train_predictions, y_train.to_numpy().ravel(), average = 'macro'))
print("MNBC Model Test F1 Score :",f1_score(mnb_test_predictions, y_test.to_numpy().ravel(), average = 'macro'))

print("MNBC Confusion matrix:\n{}".format(confusion_matrix(y_pred=y_test.to_numpy().ravel(),y_true= mnb_test_predictions)))
print("MNBC Classification report:\n",classification_report(y_pred=mnb_test_predictions,  y_true= y_test.to_numpy().ravel()))

MNBC Model Train Accuracy Score : 0.7614899555494298
MNBC Model Test Accuracy Score : 0.5992539301891819
MNBC Model Train F1 Score : 0.7589434104347412
MNBC Model Test F1 Score : 0.3278294100896285
MNBC Confusion matrix:
[[ 134   27   78  117  619]
 [  38   93   94  172  539]
 [  46   66  299  434  979]
 [  40   54  258  986 2649]
 [  35   26  148 1101 9733]]
MNBC Classification report:
               precision    recall  f1-score   support

           1       0.14      0.46      0.21       293
           2       0.10      0.35      0.15       266
           3       0.16      0.34      0.22       877
           4       0.25      0.35      0.29      2810
           5       0.88      0.67      0.76     14519

    accuracy                           0.60     18765
   macro avg       0.31      0.43      0.33     18765
weighted avg       0.73      0.60      0.65     18765



### Dataset for Stacking Classifier

In [20]:
def get_predict_probas(data):
    mlp_probas = best_model.predict(data)
    rfc_probas = rfcrs.best_estimator_.predict_proba(data)
    mnbc_probas = mnbcgs.best_estimator_.predict_proba(data)
    mlp_probas_df = pd.DataFrame(mlp_probas,columns=['mlp_1','mlp_2','mlp_3','mlp_4','mlp_5'])
    rfc_probas_df = pd.DataFrame(rfc_probas,columns=['rfc_1','rfc_2','rfc_3','rfc_4','rfc_5'])
    mnbc_probas_df = pd.DataFrame(mnbc_probas,columns=['mnbc_1','mnbc_2','mnbc_3','mnbc_4','mnbc_5'])
    return pd.concat([mlp_probas_df,rfc_probas_df,mnbc_probas_df],axis=1)

x_trainvec_stack = get_predict_probas(x_trainvec)
x_testvec_stack = get_predict_probas(x_testvec)

print(x_trainvec_stack.head())

    

    

          mlp_1         mlp_2     mlp_3     mlp_4     mlp_5     rfc_1  \
0  9.440866e-07  2.052553e-02  0.001390  0.872574  0.105510  0.177120   
1  8.284534e-12  1.081658e-17  0.000001  0.000219  0.999780  0.178567   
2  2.259267e-08  4.059383e-09  0.000087  0.081596  0.918317  0.181827   
3  1.164707e-07  5.190757e-10  0.000001  0.402942  0.597057  0.160321   
4  1.406019e-08  1.165649e-08  0.025341  0.248463  0.726196  0.150885   

      rfc_2     rfc_3     rfc_4     rfc_5    mnbc_1    mnbc_2    mnbc_3  \
0  0.161457  0.212371  0.234206  0.214846  0.112005  0.188914  0.256562   
1  0.115013  0.161904  0.204422  0.340094  0.085507  0.049869  0.127573   
2  0.147805  0.191284  0.223905  0.255180  0.100017  0.025532  0.202211   
3  0.157804  0.181526  0.223349  0.277000  0.057670  0.146505  0.181559   
4  0.132735  0.206625  0.249564  0.260191  0.030134  0.064654  0.164004   

     mnbc_4    mnbc_5  
0  0.274185  0.168334  
1  0.128682  0.608368  
2  0.252664  0.419576  
3  0.241583  0

### Stacking Classifier(Logistic Regression)

In [21]:
from sklearn.linear_model import LogisticRegression

sclf = LogisticRegression(random_state=0)

sclf_param_grid = {
    'sclf__solver' : ['newton-cg', 'lbfgs'],
    'sclf__C': [100, 10, 1.0, 0.1, 0.01]
}

imba_pipe_stack = Pipeline([
    ('sampling', RandomOverSampler(random_state=0)), 
    ('sclf', sclf)
    ])


sclfrs = HalvingGridSearchCV(estimator=imba_pipe_stack , param_grid=sclf_param_grid, factor=5, cv=5, scoring=f1, verbose=5)

sclfrs.fit(x_trainvec_stack, y_train.to_numpy().ravel())

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 58042
max_resources_: 290210
aggressive_elimination: False
factor: 5
----------
iter: 0
n_candidates: 10
n_resources: 58042
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END sclf__C=100, sclf__solver=newton-cg;, score=(train=0.965, test=0.962) total time=   6.3s
[CV 2/5] END sclf__C=100, sclf__solver=newton-cg;, score=(train=0.963, test=0.963) total time=   6.3s
[CV 3/5] END sclf__C=100, sclf__solver=newton-cg;, score=(train=0.964, test=0.964) total time=   5.7s
[CV 4/5] END sclf__C=100, sclf__solver=newton-cg;, score=(train=0.963, test=0.964) total time=   6.5s
[CV 5/5] END sclf__C=100, sclf__solver=newton-cg;, score=(train=0.963, test=0.963) total time=   6.7s


c:\Users\Lim Jia Hui\Desktop\Python\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5] END sclf__C=100, sclf__solver=lbfgs;, score=(train=0.965, test=0.962) total time=   1.1s


c:\Users\Lim Jia Hui\Desktop\Python\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5] END sclf__C=100, sclf__solver=lbfgs;, score=(train=0.963, test=0.962) total time=   1.1s


...

c:\Users\Lim Jia Hui\Desktop\Python\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 4/5] END sclf__C=100, sclf__solver=lbfgs;, score=(train=0.963, test=0.964) total time=   4.3s


c:\Users\Lim Jia Hui\Desktop\Python\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5] END sclf__C=100, sclf__solver=lbfgs;, score=(train=0.964, test=0.963) total time=   4.3s


HalvingGridSearchCV(estimator=Pipeline(steps=[('sampling',
                                               RandomOverSampler(random_state=0)),
                                              ('sclf',
                                               LogisticRegression(random_state=0))]),
                    factor=5,
                    param_grid={'sclf__C': [100, 10, 1.0, 0.1, 0.01],
                                'sclf__solver': ['newton-cg', 'lbfgs']},
                    scoring=make_scorer(f1_score, average=weighted), verbose=5)

## Evaluating

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# I probably shldv written a function for this...
sclfrs_train_predictions = sclfrs.predict(x_trainvec_stack)
sclfrs_test_predictions = sclfrs.predict(x_testvec_stack)

In [23]:
print("MLP Test Accuracy Score :",accuracy_score(y_pred=mlp_test_predictions, y_true= y_test.to_numpy().ravel()))
print("RFC Test Accuracy Score :",accuracy_score(y_pred=rfc_test_predictions, y_true= y_test.to_numpy().ravel()))
print("MNBC Model Test Accuracy Score :",accuracy_score(mnb_test_predictions, y_test.to_numpy().ravel()))
print("Stacked Model Test Accuracy Score :",accuracy_score(sclfrs_test_predictions, y_test.to_numpy().ravel()))

MLP Test Accuracy Score : 0.6490807354116707
RFC Test Accuracy Score : 0.7552358113509192
MNBC Model Test Accuracy Score : 0.5992539301891819
Stacked Model Test Accuracy Score : 0.6858513189448441


In [24]:
print("MLP Test F1 Score :",f1_score(y_pred=mlp_test_predictions, y_true=y_test.to_numpy().ravel(), average = 'macro'))
print("RFC Test F1 Score :",f1_score(y_pred=rfc_test_predictions, y_true=y_test.to_numpy().ravel(), average = 'macro'))
print("MNBC Model Test F1 Score :",f1_score(mnb_test_predictions, y_test.to_numpy().ravel(), average = 'macro'))
print("Stacked Model Test F1 Score :",f1_score(sclfrs_test_predictions, y_test.to_numpy().ravel(),average='macro'))

MLP Test F1 Score : 0.36164370916557304
RFC Test F1 Score : 0.4101705085267285
MNBC Model Test F1 Score : 0.3278294100896285
Stacked Model Test F1 Score : 0.3777871514298516


In [25]:

print("MLP Confusion matrix:\n{}".format(confusion_matrix(y_pred=y_test.to_numpy().ravel(),y_true= mlp_test_predictions)))
print("RFC Confusion matrix:\n{}".format(confusion_matrix(y_pred=y_test.to_numpy().ravel(),y_true= rfc_test_predictions)))
print("MNBC Confusion matrix:\n{}".format(confusion_matrix(y_pred=y_test.to_numpy().ravel(),y_true= mnb_test_predictions)))
print("Stacked Confusion matrix:\n{}".format(confusion_matrix(y_pred=y_test.to_numpy().ravel(),y_true= sclfrs_test_predictions)))

MLP Confusion matrix:
[[  112    31    54    58   205]
 [   34    56    53    51    97]
 [   45    73   262   312   607]
 [   28    41   216   936  2796]
 [   74    65   292  1453 10814]]
RFC Confusion matrix:
[[   99    20    28    19   109]
 [    7    37     7     1     3]
 [   13    40   186    96   130]
 [   19    45   233   870  1297]
 [  155   124   423  1824 12980]]
MNBC Confusion matrix:
[[ 134   27   78  117  619]
 [  38   93   94  172  539]
 [  46   66  299  434  979]
 [  40   54  258  986 2649]
 [  35   26  148 1101 9733]]
Stacked Confusion matrix:
[[  114    25    42    50   179]
 [   34    64    62    49    97]
 [   42    67   252   287   554]
 [   25    35   184   817  2066]
 [   78    75   337  1607 11623]]


In [26]:

print("MLP Classification report:\n",classification_report(y_pred=mlp_test_predictions,  y_true= y_test.to_numpy().ravel()))
print("RFC Classification report:\n",classification_report(y_pred=rfc_test_predictions,  y_true= y_test.to_numpy().ravel()))

MLP Classification report:
               precision    recall  f1-score   support

           1       0.24      0.38      0.30       293
           2       0.19      0.21      0.20       266
           3       0.20      0.30      0.24       877
           4       0.23      0.33      0.27      2810
           5       0.85      0.74      0.79     14519

    accuracy                           0.65     18765
   macro avg       0.34      0.39      0.36     18765
weighted avg       0.71      0.65      0.67     18765

RFC Classification report:
               precision    recall  f1-score   support

           1       0.36      0.34      0.35       293
           2       0.67      0.14      0.23       266
           3       0.40      0.21      0.28       877
           4       0.35      0.31      0.33      2810
           5       0.84      0.89      0.86     14519

    accuracy                           0.76     18765
   macro avg       0.52      0.38      0.41     18765
weighted avg       0.

In [27]:
print("MNBC Classification report:\n",classification_report(y_pred=mnb_test_predictions,  y_true= y_test.to_numpy().ravel()))
print("Stacked Classification report:\n",classification_report(y_pred=sclfrs_test_predictions,  y_true= y_test.to_numpy().ravel()))

MNBC Classification report:
               precision    recall  f1-score   support

           1       0.14      0.46      0.21       293
           2       0.10      0.35      0.15       266
           3       0.16      0.34      0.22       877
           4       0.25      0.35      0.29      2810
           5       0.88      0.67      0.76     14519

    accuracy                           0.60     18765
   macro avg       0.31      0.43      0.33     18765
weighted avg       0.73      0.60      0.65     18765

Stacked Classification report:
               precision    recall  f1-score   support

           1       0.28      0.39      0.32       293
           2       0.21      0.24      0.22       266
           3       0.21      0.29      0.24       877
           4       0.26      0.29      0.28      2810
           5       0.85      0.80      0.82     14519

    accuracy                           0.69     18765
   macro avg       0.36      0.40      0.38     18765
weighted avg    